In [1]:
pwd

'/home/aryan-dhanuka/End_to_End_Chest_Cancer_Detection_ML_Project_using_DVC/research'

In [2]:
import os
os.chdir('../')

In [3]:
pwd

'/home/aryan-dhanuka/End_to_End_Chest_Cancer_Detection_ML_Project_using_DVC'

In [4]:
import os
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


# --------------------------
# Load VGG16 for Feature Extraction
# --------------------------
vgg = VGG16(weights='imagenet', include_top=False, pooling='avg')

# --------------------------
# Load Dataset
# --------------------------
def load_data(data_dir):
    X, Y = [], []
    class_names = os.listdir(data_dir)
    label_map = {cls: idx for idx, cls in enumerate(class_names)}

    for cls in class_names:
        class_path = os.path.join(data_dir, cls)
        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)

            img = load_img(img_path, target_size=(224, 224))
            img = img_to_array(img)
            img = preprocess_input(img)

            X.append(img)
            Y.append(label_map[cls])

    return np.array(X), np.array(Y), label_map

data_dir = "artifacts/data_ingestion/Data/train"
X, Y, label_map = load_data(data_dir)

# --------------------------
# Feature Extraction
# --------------------------
features = vgg.predict(X, verbose=1)  # shape → (num_samples, 512)

X_train, X_test, y_train, y_test = train_test_split(
    features, Y, test_size=0.2, random_state=42
)

print("Feature shape:", X_train.shape)


2025-11-17 20:01:31.667448: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-17 20:01:31.694345: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-17 20:01:32.906956: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-11-17 20:01:32.926394:

20/20 [==============================] - 6s 122ms/step
Feature shape: (490, 512)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=20,
    random_state=42
)

rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


CNN  Accuracy: 0.8048780487804879
[[14  0  1 12]
 [ 0 23  0  8]
 [ 0  0 29  0]
 [ 0  1  2 33]]


In [ ]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)

print("Naive Bayes Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Naive Bayes Accuracy: 0.7479674796747967
              precision    recall  f1-score   support

           0       0.89      0.63      0.74        27
           1       0.73      0.71      0.72        31
           2       0.88      0.97      0.92        29
           3       0.60      0.69      0.64        36

    accuracy                           0.75       123
   macro avg       0.77      0.75      0.75       123
weighted avg       0.76      0.75      0.75       123



In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

dt = DecisionTreeClassifier(
    max_depth=5,          # shallow tree → weaker performance
    min_samples_split=10,
    random_state=42
)

dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)

print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Decision Tree Accuracy: 0.6097560975609756
              precision    recall  f1-score   support

           0       0.53      0.37      0.43        27
           1       0.54      0.61      0.58        31
           2       0.89      0.83      0.86        29
           3       0.52      0.61      0.56        36

    accuracy                           0.61       123
   macro avg       0.62      0.61      0.61       123
weighted avg       0.62      0.61      0.61       123

